In [179]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np
import sys
sys.path.insert(0, '..')
from Common import preprocessing,evaluation
classes_map = {'DOC':1, 'ENTER':2, 'ORG':3, 'PRIV':4, 'RANG':5, 'HOST':6}

In [60]:
df = pd.read_csv('..//Data//labeled.abiturient.ru.txt', delimiter=';', engine='python',encoding='utf8')

In [61]:
questions = np.array(df.question)
questions = preprocessing.preprocess_list(questions)

In [201]:
vectorizer = TfidfVectorizer(ngram_range=(1,1))
X = vectorizer.fit_transform(questions)

In [202]:
classes = np.array(df['class'])
y = list(map(lambda x: classes_map[x],classes))

In [203]:
log_reg = OneVsRestClassifier(LogisticRegression(random_state=0,C=10,solver='lbfgs',)).fit(X, y)

In [204]:
def predict (question,model):
    question = preprocessing.preprocess_list([question])[0]
    vect = vectorizer.transform([question])
    return model.predict_proba(vect)

In [205]:
print(predict("Когда можно донести документы ?",log_reg))

[[0.81185057 0.02685152 0.09852576 0.00802609 0.03660995 0.01813611]]


In [206]:
df_test = pd.read_csv('..//Data//labeled.pstu.ru.txt', delimiter=';', engine='python',encoding='utf8')
test_questions = np.array(df_test.question)
test_questions = preprocessing.preprocess_list(test_questions)
X_test = vectorizer.transform(test_questions)
test_classes = np.array(df_test['class'])
y_test = list(map(lambda x: classes_map[x],test_classes))

In [190]:
evaluation.get_CV_scores(log_reg,X_test,y_test,'f1_weighted').mean()

c:\users\aleksandr.perevalov\appdata\local\continuum\miniconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\aleksandr.perevalov\appdata\local\continuum\miniconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\aleksandr.perevalov\appdata\local\continuum\miniconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.7207031105230748

In [26]:
#model = KeyedVectors.load_word2vec_format('..//..//all.norm-sz100-w10-cb0-it1-min100.w2v',binary=True,unicode_errors='ignore')

In [219]:
for i in sorted(list(zip(log_reg.estimators_[0].coef_[0],vectorizer.get_feature_names()))):
    print(i)

(-3.7744394092258218, 'список')
(-3.658744690192521, 'общежитие')
(-3.465352527357196, 'балл')
(-3.3942961418952833, 'поступить')
(-2.8806116084367437, 'направление')
(-2.700674868477908, 'экзамен')
(-2.465842415397171, 'проходить')
(-2.272028660453989, 'набор')
(-2.27142749697732, 'работать')
(-2.238802138852376, 'обучение')
(-2.1832931561834457, 'заселение')
(-2.014158251282766, 'сдавать')
(-2.010792683074339, 'этмый')
(-2.009895962682564, 'зачислить')
(-1.9209352915742304, 'вступительный')
(-1.8737167282073706, 'предмет')
(-1.7937138350138595, 'платный')
(-1.7810015019355707, 'бюджетный')
(-1.7698674839903863, 'конкурс')
(-1.7327567851024797, 'год')
(-1.716859432308257, 'стоить')
(-1.7090263248720017, 'экт')
(-1.7039078463325368, 'стипендия')
(-1.6628650440480344, 'учиться')
(-1.637350724809186, 'общий')
(-1.6087073631114077, 'проходная')
(-1.5311081096267236, 'проводиться')
(-1.5246111261106743, 'поступать')
(-1.520612382611132, 'приказ')
(-1.513633448270051, 'суббота')
(-1.5076670

(1.7290831726311098, 'подсказать')
(1.7939218902667735, 'нести')
(1.7950904752207286, 'индекс')
(1.7951252077066886, 'подлинник')
(1.8044583440181674, 'приём')
(1.8214258901319842, 'ничто')
(1.8292916499951328, 'долго')
(1.8672430169849037, 'лайна')
(1.9523806834587554, 'поступление')
(1.9704036179979634, 'вид')
(1.9907887423860489, 'возможность')
(2.0948890745414532, 'срок')
(2.0959323183352536, 'инна')
(2.096746494428741, 'заполнять')
(2.1052269080017183, 'предоставлять')
(2.1723697841978704, 'обязательно')
(2.1739898739085577, 'лично')
(2.186832210325359, 'заверять')
(2.2640392616360288, 'повлиять')
(2.2658067453269344, 'подача')
(2.269192264182601, 'заполнение')
(2.374606386711682, 'ещё')
(2.4317897248330524, 'аттестат')
(2.437993498647803, 'образец')
(2.4472600139419045, 'идти')
(2.4793628251982436, 'посылать')
(2.486674007764819, 'страница')
(2.4879035938603824, 'форма')
(2.5448374796704667, 'оригинал')
(2.5664103186804312, 'егэ')
(2.650085939117127, 'номер')
(2.654178031719739, 